<a href="https://colab.research.google.com/github/yipclara/spotify-wrapped/blob/main/spotify_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd
import json
import requests
import re
import ast
import os
from dotenv import load_dotenv
from matplotlib import pyplot as plt
import seaborn as sns

# Load variables from the .env file

load_dotenv()

# Access the credentials using os.getenv()
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

#### loading local file

In [ ]:
# load streaming json into df
with open('StreamingHistory_2025.json', 'r') as file:
    data = json.load(file)

sh = pd.DataFrame.from_dict(data)
sh['endTime'] = pd.to_datetime(sh['endTime'])
sh

## Things I want to know
### by time
1. split into month columns
1. split into date columns

- minutes per year
- minutes per month (in hours)
- most listened day of the year

### by songs
- num unique songs
- all songs by minutes listened (in hours)
- all songs by instances listened

### by artist
- num unique artists
- all artists by minutes listened
- top 5most listened to artist by week/month
   - graph
   - see if it correlates with concerts

### genres
1. do some bucketing first (manual)
   1. dupe rows with multiple genres??
   1. so that you can count distincts, but thats the only use for this dataset
   1. and then do the theming, add a dict -> update columns
1. try to ensure each song is only one theme
1. 1 song -> 1 theme

- top genres in minutes (hours)
   - by theme
- top genres per week/month
   - by theme
- most fun/unique genre (manual inspection)






### Getting data

In [ ]:
# Spotify api https://developer.spotify.com/documentation/web-api
def get_access_token():
    response = requests.post(
        "https://accounts.spotify.com/api/token",
        data={"grant_type": "client_credentials",
              "client_id": client_id,
              "client_secret": client_secret},
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    print(response)
    return response.json()["access_token"]


token = get_access_token()
print(token)

In [ ]:
# load prefetched artists
with open('data/artists.csv', 'r') as file:
    reader = csv.DictReader(file)
    artists = {row['artist_name']: {'artist_id': row['artist_id'], 'genre':row['genre']} for row in reader}
# artists

In [ ]:
def get_artist_and_genre(artist_name):
    # print(f"searching for artist {artist_name}")

    if artist_name in artists.keys():
      return artists[artist_name]['artist_id'], artists[artist_name]['genre']

    # else, make api call 

    # handle special characters
    artist_name_escaped = artist_name.replace("&", "%26")

    response = requests.get(
        f"https://api.spotify.com/v1/search?q={artist_name_escaped}&type=artist",
        headers={"Authorization": f"Bearer {token}",
                 "Content-Type": "application/x-www-form-urlencoded"}
    )
    if response.status_code == 200:
      items = response.json()["artists"]["items"]
      if len(items) > 0:
        artist_id = response.json()["artists"]["items"][0]["id"]
        genre = response.json()["artists"]["items"][0]["genres"]

        # write to artists dict
        artists[artist_name] = {
          "artist_id": artist_id,
          "genre": genre
        }

        return artist_id, genre
      else:
        print(f"No items found in api response for artist {artist_name}")
        return None, None
    else:
      print(f"Unexpected error: {response.text}")
      return None, None

In [ ]:
unique_artists = sh["artistName"].unique()

for artistName in unique_artists:
  aID, aGenre = get_artist_and_genre(artistName)

# save artists dict to csv
with open('data/artists.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['artist_name', 'artist_id', 'genre'])
    for artist, info in artists.items():
      writer.writerow([artist, info['artist_id'], info['genre']])

In [ ]:
# add genres manually if needed
for artist in artists:
    if artists[artist]['genre'] == []:
        print(f"artist {artist} has no genre, please add manually")

In [ ]:
# reload the artists csv after adding genres manually
with open('data/artists.csv', 'r') as file:
    reader = csv.DictReader(file)
    artists = {row['artist_name']: {'artist_id': row['artist_id'], 'genre':row['genre']} for row in reader}
artists


In [ ]:
# add genres to sh 
# given sh doenst have a genre column yet, and all the artists (if found) exist and have genres in artist.csv
sh['genres'] = sh['artistName'].map(lambda x: artists.get(x, {}).get('genre', 'Unknown'))
sh

### TIME

#### Getting data

In [ ]:
def get_month(row):
  return row["endTime"].to_pydatetime().month

def get_week(row):
  return row["endTime"].to_pydatetime().isocalendar().week

def convert_ts_to_date(row):
  return row["endTime"].date()

In [ ]:
sh["month"] = sh.apply(get_month, axis=1)
sh["week"] = sh.apply(get_week, axis=1)
sh["date"] = sh.apply(convert_ts_to_date, axis=1)
# sh

#### Analyzing

In [ ]:
# listening time by month
def convert_to_hours(row):
  rawHours = (row["msPlayed"]/3600000)
  return round(rawHours, 2)

def convert_to_minutes(row):
  rawMins = (row["msPlayed"]/60000)
  return round(rawMins, 2)

In [ ]:
listeningByMonth = sh.groupby('month', as_index=False, sort=False).agg({'msPlayed':'sum'})
listeningByMonth["hours"] = listeningByMonth.apply(convert_to_hours, axis=1)
listeningByMonth

In [ ]:
# @title month vs hours
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.color_palette('tab10'))
  xs = series['month']
  ys = series['hours']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = listeningByMonth.sort_values('month', ascending=True)
_plot_series(df_sorted, '')
ax.set_ylim(bottom=0)
sns.despine(fig=fig, ax=ax)
plt.xlabel('month')
_ = plt.ylabel('hours')

In [ ]:
# grab the most listened month
listeningByMonth.sort_values(by=['msPlayed'],ascending=False)

In [ ]:
# listening time by week
listeningByWeek = sh.groupby('week', as_index=False, sort=False).agg({'msPlayed':'sum'})
listeningByWeek["hours"] = listeningByWeek.apply(convert_to_hours, axis=1)
listeningByWeek

In [ ]:
# @title date vs hours
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['week']
  ys = series['hours']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = listeningByWeek.sort_values('week', ascending=True)
_plot_series(df_sorted, '')
ax.set_ylim(bottom=0)
sns.despine(fig=fig, ax=ax)
plt.xlabel('week')
_ = plt.ylabel('hours')

In [ ]:
# grab the most listened to week
listeningByWeek.sort_values(by=['msPlayed'],ascending=False)

In [ ]:
# listening time by date
listeningByDate = sh.groupby('date', as_index=False, sort=False).agg({'msPlayed':'sum'})
listeningByDate["minutes"] = listeningByDate.apply(convert_to_minutes, axis=1)
listeningByDate["hours"] = listeningByDate.apply(convert_to_hours, axis=1)
listeningByDate

In [ ]:
# grab the most listened to date
listeningByDate.sort_values(by=['msPlayed'],ascending=False)

### SONGS

In [ ]:
# uncomment this cell if you want to save off your listens in desc frequency order
# # songs by instances listened (distinct obvi)
# sh.groupby(['artistName', 'trackName']).size().reset_index().rename(columns={0:'count'}).sort_values(by=['count'],ascending=False)
# .to_csv("data/listens.csv", index=False)

In [ ]:
# songs by hours listened
listeningBySong = sh.groupby(['artistName', 'trackName'], as_index=False, sort=False).agg({'msPlayed':'sum'})
listeningBySong["hours"] = listeningBySong.apply(convert_to_hours, axis=1)
listeningBySong.sort_values(by=['msPlayed'],ascending=False)

### ARTIST

In [ ]:
# artists by minutes/hours listened
listeningByArtist = listeningBySong.groupby(['artistName'], as_index=False, sort=False).agg({'msPlayed':'sum'})
listeningByArtist["hours"] = listeningByArtist.apply(convert_to_hours, axis=1)
listeningByArtist.sort_values(by=['hours'],ascending=False)

In [ ]:
# listening by month/artist
## agg by month, artist
listeningByMonthArtist = sh.groupby(['month', 'artistName'], as_index=False, sort=False).agg({'msPlayed':'sum'})
listeningByMonthArtist["minutes"] = listeningByMonthArtist.apply(convert_to_minutes, axis=1)
listeningByMonthArtist

In [ ]:
## (partition by) per month, grab the top 3
topArtistByMonth = listeningByMonthArtist.groupby(['month'])[['month', 'artistName', 'minutes']].apply(lambda grp: grp.nlargest(3, 'minutes'))
topArtistByMonth.to_csv("data/topMonthlyArtists2025.csv", index=False)

In [ ]:
sns.catplot(
    x="month",       # x variable name
    y="minutes",       # y variable name
    hue="artistName",  # group variable name
    data=topArtistByMonth,     # dataframe to plot
    kind="point",
    palette=sns.color_palette('tab10'),
    aspect=3,
)

### GENRES

#### Getting data

In [ ]:
# get all unique microgenres
microgenres = set()
for index, row in sh.iterrows():
  if not pd.isna(row['genres']):
    gs = row['genres'][1:-1].split(",")
    for mg in gs:
      # print(mg)
      microgenres.add(mg.strip())

print(microgenres)

In [ ]:
# load the csv of the mappings
genres = pd.read_csv('genreTags.csv')

def lookup_genre(row):
  x = re.search("\[\'(.)*\'\]", row['genres'])
  if x:
    x = ast.literal_eval(x.group())
    resultsLen = len(genres.loc[genres['tag'] == "'" + x[0] + "'"]['genre'].values)

    if resultsLen > 0:
      y = genres.loc[genres['tag'] == "'" + x[0] + "'"]['genre'].values[0]
      return y
    elif resultsLen == 0:
      return None
    else:
      print(row['genres'])
      return None
  else:
    if not row['genres']:
      print(row['genres'])
    return None
  
sh["theme"] = sh.apply(lookup_genre, axis=1)

In [ ]:
# top most listened artists with no theme, to help with manual tagging of themes for artists that dont have a theme but are listened to a lot
missing_theme_artists = sh[sh['theme'].isnull()].groupby(['artistName'], as_index=False, sort=False).agg({'msPlayed':'sum'})
missing_theme_artists["hours"] = missing_theme_artists.apply(convert_to_hours, axis=1)
missing_theme_artists.sort_values(by=['hours'], ascending=False).head(20)

#### Analyzing

In [ ]:
# genres by minutes/hours listened
listeningByGenre = sh.groupby(['theme'], as_index=False, sort=False, dropna=False).agg({'msPlayed':'sum'})
listeningByGenre["hours"] = listeningByGenre.apply(convert_to_hours, axis=1)
listeningByGenre.sort_values(by=['hours'],ascending=False)

In [ ]:
# listening by month/artist

## agg by month, artist
listeningByMonthGenre = sh.groupby(['month', 'theme'], as_index=False, sort=False, dropna=False).agg({'msPlayed':'sum'})
listeningByMonthGenre["minutes"] = listeningByMonthGenre.apply(convert_to_minutes, axis=1)
listeningByMonthGenre

In [ ]:
## (partition by) per month, grab the top 10
topGenreByMonth = listeningByMonthGenre.groupby(['month'], dropna=False)[['month', 'theme', 'minutes']].apply(lambda grp: grp.nlargest(5, 'minutes'))
topGenreByMonth

In [ ]:
# top genres
topGenres = topGenreByMonth["theme"].unique()
topGenres

# basically my top 10 genres

In [ ]:
# listening of top 10 genres
top10Genres = listeningByGenre.sort_values(['hours'], ascending=False).head(10)

fig, ax = plt.subplots()
ax.pie(top10Genres['hours'], labels=top10Genres['theme'], colors=sns.color_palette('tab10'))